# 🧩 Online Retail II ETL Project

**Goal:** Build a complete ETL process for a retail dataset (UCI Online Retail II).  
**Tools:** Python (Pandas) → Cleaned CSV → Power BI Dashboard  
**Output:** Ready-to-use dataset for BI dashboards.

# --- 1. Import Libraries ---

In [1]:
import pandas as pd
from pathlib import Path

# --- 2. Define Paths ---

In [2]:
BASE_DIR = Path.cwd().parent if Path.cwd().name == 'etl' else Path.cwd()
RAW_DIR = BASE_DIR / "data" / "raw"
PROCESSED_DIR = BASE_DIR / "data" / "processed"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

In [3]:
print("📂 Raw:", RAW_DIR)
print("📂 Processed:", PROCESSED_DIR)

📂 Raw: C:\Users\venzo\retail-etl-powerbi\data\raw
📂 Processed: C:\Users\venzo\retail-etl-powerbi\data\processed


# --- 3. Load the Excel file ---
The dataset has two sheets: 'Year 2009-2010' and 'Year 2010-2011'
You can load one or merge both

In [4]:
file_path = RAW_DIR / "online_retail_II.xlsx"

In [5]:
df1 = pd.read_excel(file_path, sheet_name="Year 2009-2010")
df2 = pd.read_excel(file_path, sheet_name="Year 2010-2011")

Combine both years

In [6]:
df = pd.concat([df1, df2], ignore_index=True)

In [7]:
print("Datos combinados:", df.shape)

Datos combinados: (1067371, 8)


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Invoice      1067371 non-null  object        
 1   StockCode    1067371 non-null  object        
 2   Description  1062989 non-null  object        
 3   Quantity     1067371 non-null  int64         
 4   InvoiceDate  1067371 non-null  datetime64[ns]
 5   Price        1067371 non-null  float64       
 6   Customer ID  824364 non-null   float64       
 7   Country      1067371 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 65.1+ MB


In [9]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


# --- 3b. Inspect raw columns (quick sanity check) ---

In [10]:
print(df.columns.tolist())

['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country']


# --- 3c. Standardize column names to a common schema ---

Online Retail II often has: 'Invoice', 'StockCode', 'Description', 'Quantity','InvoiceDate', 'Price', 'Customer ID', 'Country'

In [11]:
def standardize_columns(df):
    # Trim whitespace from headers
    df = df.rename(columns=lambda c: c.strip())

    # Build a flexible rename map
    rename_map = {}
    cols = set(df.columns)

    if "Price" in cols:
        rename_map["Price"] = "UnitPrice"
    # keep UnitPrice if it already exists (Retail I)

    if "Invoice" in cols:
        rename_map["Invoice"] = "InvoiceNo"

    if "Customer ID" in cols:
        rename_map["Customer ID"] = "CustomerID"
    elif "CustomerID" in cols:
        rename_map["CustomerID"] = "CustomerID"  # no-op, just explicit

    # Apply renames
    df = df.rename(columns=rename_map)

    # Ensure required columns exist
    required = ["InvoiceNo","StockCode","Description","Quantity","InvoiceDate","UnitPrice","CustomerID","Country"]
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns after rename: {missing}. "
                         f"Got: {df.columns.tolist()}")

    return df

df = standardize_columns(df)
print("✅ Columns after standardization:", df.columns.tolist())

✅ Columns after standardization: ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']


# --- 4. Quick Exploration ---

In [12]:
print("Filas y columnas:", df.shape)
print("\nValores nulos por columna:\n", df.isna().sum())
print("\nValores únicos por columna:\n", df.nunique())


Filas y columnas: (1067371, 8)

Valores nulos por columna:
 InvoiceNo           0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     243007
Country             0
dtype: int64

Valores únicos por columna:
 InvoiceNo      53628
StockCode       5305
Description     5698
Quantity        1057
InvoiceDate    47635
UnitPrice       2807
CustomerID      5942
Country           43
dtype: int64


# --- 5. Data Cleaning ---

In [13]:
clean_df = df.copy()

In [14]:
# Types
clean_df["Quantity"] = pd.to_numeric(clean_df["Quantity"], errors="coerce")
clean_df["UnitPrice"] = pd.to_numeric(clean_df["UnitPrice"], errors="coerce")
clean_df["InvoiceDate"] = pd.to_datetime(clean_df["InvoiceDate"], errors="coerce")

In [15]:
# Remove exact duplicates
clean_df = clean_df.drop_duplicates()

In [16]:
# Create LineTotal
clean_df["LineTotal"] = clean_df["Quantity"] * clean_df["UnitPrice"]

# Identify returns (Quantity < 0 or credit invoices starting with 'C')
clean_df["IsReturn"] = (clean_df["Quantity"] < 0) | (clean_df["InvoiceNo"].astype(str).str.startswith("C", na=False))

# Remove invalid price rows (keep returns)
clean_df = clean_df[(clean_df["UnitPrice"] > 0) | (clean_df["IsReturn"])]

# Fill missing CustomerID
clean_df["CustomerID"] = clean_df["CustomerID"].fillna("Unknown")

In [17]:
print("✅ Cleaning completed. Shape:", clean_df.shape)
clean_df.head()

✅ Cleaning completed. Shape: (1030410, 10)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,LineTotal,IsReturn
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4,False
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,False
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,False
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8,False
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0,False


In [18]:
clean_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,LineTotal,IsReturn
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4,False
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,False
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,False
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8,False
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0,False


# --- 6. Feature Engineering ---

In [19]:
clean_df["InvoiceYear"] = clean_df["InvoiceDate"].dt.year
clean_df["InvoiceMonth"] = clean_df["InvoiceDate"].dt.month
clean_df["InvoiceDay"] = clean_df["InvoiceDate"].dt.day
clean_df["InvoiceDateKey"] = clean_df["InvoiceDate"].dt.strftime("%Y%m%d").astype(int)

# --- 7. Export the Clean Dataset ---

In [20]:
fact_sales_path = PROCESSED_DIR / "fact_sales.csv"
clean_df.to_csv(fact_sales_path, index=False)
print(f"✅ fact_sales.csv exported to: {fact_sales_path}")

✅ fact_sales.csv exported to: C:\Users\venzo\retail-etl-powerbi\data\processed\fact_sales.csv


# --- 8. Create Dimension Tables ---

In [21]:
dim_product = clean_df[["StockCode", "Description"]].drop_duplicates()
dim_customer = clean_df[["CustomerID", "Country"]].drop_duplicates()

dim_product.to_csv(PROCESSED_DIR / "dim_product.csv", index=False)
dim_customer.to_csv(PROCESSED_DIR / "dim_customer.csv", index=False)

print("✅ Dimension tables exported (product & customer).")

✅ Dimension tables exported (product & customer).


# --- 9. Final Check ---

In [22]:
print("\nFiles created in /data/processed/:")
for file in PROCESSED_DIR.iterdir():
    print(" -", file.name)


Files created in /data/processed/:
 - dim_customer.csv
 - dim_product.csv
 - fact_sales.csv
